In [2]:
import numpy as np
import pandas as pd
import requests
import re
import sqlite3
import time
import openai

In [3]:
db = sqlite3.connect("scraping.db")

In [4]:
def create_article_table(db):
    db.execute("drop table if exists article")

    db.execute("""
    create table if not exists article (id integer primary key, headline varchar(255), time timestamp, content text);
    """);

    db.commit()

In [5]:
# create_article_table(db)

In [6]:
def insert_article(id, title=None, timestamp=None, content=None):
    content = content.replace('"', "'") if content else None
    title = title.replace('"', "'") if title else None
    try:
        db.execute(f'insert into article values ({id}, "{title}", "{timestamp}", "{content}")');
    except sqlite3.IntegrityError as e:
        print(id, " was a duplicate")
    db.commit()

In [7]:
cur = db.execute("select * from article")
db.commit()
for row in cur:
    print(row)
    break

(848394, 'None', '2016-05-24 20:36:29', 'None')


In [10]:
db.commit()
del db

NameError: name 'db' is not defined

In [10]:
url = "https://armenpress.am/eng/ajax/"
method = "POST"

def get_article_list(id):
    form_data = {
        "xdo": "news",
        "lastid": id
    }

    resp = requests.post(url, form_data)
    (article_ids := re.findall('Article"><a href="/eng/news/(\d+).html"', resp.text))
    return [int(i) for i in article_ids]


In [11]:
lastid = 1

while len((articles := get_article_list(lastid))) > 0:
    lastid += 1
    [insert_article(i) for i in articles]
    time.sleep(0.15)
    print(lastid, end="\r")

In [11]:
article_url = "https://armenpress.am/eng/news/" + str(row[0])
resp = requests.get(article_url)

In [12]:
(title := re.findall('<h1 class="opennewstitle" itemprop="name">(.+)</h1>', resp.text)[0])
(content := re.findall('itemprop="articleBody"><p>(.+)</p>', resp.text)[0])


'Sergey Lavrov says Russia positively assesses the prospects of CIS'

In [13]:
article_url

'https://armenpress.am/eng/news/591925'

In [26]:
form_data = {
    "xdo": "news",
    "lastid": 1
}

resp = requests.post(url, form_data)

In [1]:
content

NameError: name 'content' is not defined

In [21]:
print(resp.text)

<!doctype html>
<html  lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://opengraphprotocol.org/schema/" itemscope itemtype="http://schema.org/WebPage">
<head>
<meta charset="utf-8">
<title>Sergey Lavrov says Russia positively assesses the prospects of CIS | ARMENPRESS Armenian News Agency </title>
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">
<meta name="description" itemprop="description" content="MOSCOW, FEBRUARY 25, ARMENPRESS: Russia positively assesses the prospects of CIS, Russian Foreign Minister Sergey Lavrov said at a press conference, adding that from its very beginning the financial crisis brought CIS countries to the point of realizing the power of operating together. Mr. Lavrov said the interest of developing the financial part of CIS has increased among the member-states during the last 2-3 years.   Sergey Lavrov said since the very start of the crisis, that interest has become more practical. &amp;ldquo;There appeared